In [7]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Create the connection
dt = sqlite3.connect(r'database.sqlite')

# create the dataframe from a query
df = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type ='table'", dt)

In [8]:
#Lets look at the features in our dataset
df

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [11]:
#List of the countries
countries = pd.read_sql("SELECT * FROM Country; ", dt)
countries

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [14]:
#For our first analytics, lets look at the goals score by the top nations and their league
leagues = pd.read_sql("SELECT * From League JOIN Country on Country.id = League.country_id; ", dt)
leagues

,id,country_id,name,id,name
0,1,1,Belgium Jupiler League,1,Belgium
1,1729,1729,England Premier League,1729,England
2,4769,4769,France Ligue 1,4769,France
3,7809,7809,Germany 1. Bundesliga,7809,Germany
4,10257,10257,Italy Serie A,10257,Italy
5,13274,13274,Netherlands Eredivisie,13274,Netherlands
6,15722,15722,Poland Ekstraklasa,15722,Poland
7,17642,17642,Portugal Liga ZON Sagres,17642,Portugal
8,19694,19694,Scotland Premier League,19694,Scotland
9,21518,21518,Spain LIGA BBVA,21518,Spain


In [35]:
players = pd.read_sql(""" SELECT * FROM Player JOIN Player_Attributes ON Player.player_api_id = Player_Attributes.player_api_id ORDER BY Player_Attributes.overall_rating DESC LIMIT 100 """, dt)
players

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,id,player_fifa_api_id,player_api_id,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,6176,30981,Lionel Messi,158023,1987-06-24 00:00:00,170.18,159,102483,158023,30981,...,90.0,74,13,23,21.0,6,11,15,14,8
1,6176,30981,Lionel Messi,158023,1987-06-24 00:00:00,170.18,159,102484,158023,30981,...,90.0,74,13,23,21.0,6,11,15,14,8
2,6176,30981,Lionel Messi,158023,1987-06-24 00:00:00,170.18,159,102485,158023,30981,...,90.0,74,13,23,21.0,6,11,15,14,8
3,6176,30981,Lionel Messi,158023,1987-06-24 00:00:00,170.18,159,102492,158023,30981,...,90.0,76,25,21,20.0,6,11,15,14,8
4,6176,30981,Lionel Messi,158023,1987-06-24 00:00:00,170.18,159,102493,158023,30981,...,90.0,76,25,21,20.0,6,11,15,14,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6377,40636,Luis Suarez,176580,1987-01-24 00:00:00,182.88,187,105984,176580,40636,...,84.0,85,30,45,38.0,27,25,31,33,37
96,6377,40636,Luis Suarez,176580,1987-01-24 00:00:00,182.88,187,105985,176580,40636,...,84.0,85,30,45,38.0,27,25,31,33,37
97,6556,27299,Manuel Neuer,167495,1986-03-27 00:00:00,193.04,203,109034,167495,27299,...,70.0,37,10,10,11.0,85,87,91,90,87
98,6556,27299,Manuel Neuer,167495,1986-03-27 00:00:00,193.04,203,109035,167495,27299,...,70.0,37,10,10,11.0,85,87,91,90,87


In [36]:
# Assuming 'overall_rating' is one of the attributes
print(players[['player_name', 'overall_rating']].head())


    player_name  overall_rating
0  Lionel Messi              94
1  Lionel Messi              94
2  Lionel Messi              94
3  Lionel Messi              94
4  Lionel Messi              94
